(sec:exercise-othello)=
# 演習3 - オセロエージェントを作る

今回の演習では、前節、[基本のオセロAI](sec:othello-agent)の延長として、より強いオセロのAIを作成してみよう。

前節では、オセロAIの基本であるゲーム木の探索方法について、マス評価値に基づいた[ミニマックス探索](ssec:minimax)と、マス評価値に依らない[原始モンテカルロ探索](ssec:monte-carlo)について解説した。

本演習では、これらの延長として、Q関数の**関数近似**に基づいたAIの強化に取り組んでみよう。

In [1]:
# joblib用のtqdm
import contextlib
import joblib
from tqdm.notebook import tqdm

@contextlib.contextmanager
def tqdm_joblib(total=None, **kwargs):
    pbar = tqdm(total=total, miniters=1, smoothing=0, **kwargs)

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            pbar.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback

    try:
        yield pbar
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        pbar.close()

## Q学習における関数近似

**関数近似**とは、Q関数$Q(s, a)$を何らかの機械学習モデルによって近似する手法を指す。[Q学習の基本](sec:q-learning)で紹介した手法ではQテーブルを離散化していたが、このような手法は、状態数の多いゲームでは適用が難しい。

今回、演習で扱うオセロも取り得る盤の状態は、64個のマスについて、黒、白、空の3通りが考えられるから、その状態数は$3^{64} \approx 3 \times 10^{30}$にもなる。従って、このような大量の状態数に対して、そのまま離散的なQテーブルを定義することは現実的ではない。

そこで、状態$s$を入力とし、取り得る行動$a \in \mathcal{A}$について、行動価値を与えるような関数$Q(s, a)$を機械学習モデルによって表現することを考える。

最も単純な例として、状態を表わすパラメータをベクトル$\mathbf{s}$として表わし、この線形変換により、各行動の価値を要素に持つベクトル$\mathbf{a}$を求めるようなモデルを考えることができる。

$$
\mathbf{a} = \mathbf{W} \mathbf{s} + \mathbf{b}
$$

このような線形モデルであれば、Q学習の更新式:

$$
Q_{\rm new}(s, a) = Q(s,a ) + \alpha \left[ R(s, a) + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]
$$

の第2項に現れるTD誤差を最小化するようにモデルのパラメータ$\mathbf{W}$, $\mathbf{b}$を最適化すれば良い。

### scikit-learnを用いた関数近似

In [2]:
import othello
from othello import Move, Player

In [3]:
env = othello.make()

In [4]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor

reg = MultiOutputRegressor(SGDRegressor())
# reg = MLPRegressor(learning_rate_init=0.01)
dummy_X = [np.random.random(size=(8 * 8))]
dummy_y = [np.random.random(size=(8 * 8 + 1))]
reg.partial_fit(dummy_X, dummy_y)

In [5]:
gamma = 0.99
epsilon = 0.5

for _ in tqdm(range(100)):
    X = []
    y = []
    player, board = env.reset()
    while not env.gameset():
        # 入力のベクトルは現在の状態
        feature = player * board.reshape((-1))
        X.append(feature.copy())
    
        # 手を指して状態を進める
        # 以下はε-greedy法で手を選ぶ
        moves = env.legal_moves(player)
        if len(moves) == 0:
            move = Move.Pass(player)
        elif np.random.uniform(0.0, 1.0) < epsilon:
            move = np.random.choice(moves)
        else:
            action_values = reg.predict([feature])[0]
            best_move = None
            best_score = -np.inf
            for m in moves:
                action_index = m.y * 8 + m.x
                if best_score < action_values[action_index]:
                    best_score = action_values[action_index]
                    best_move = m
            move = best_move
                
        next_player, next_board = env.step(move)
    
        # 対戦結果に応じて報酬(罰)を与える
        reward = 0.0
        if env.gameset():
            n_me = env.count(player)
            n_you = env.count(next_player)
            reward = n_me - n_you          
    
        # 次の状態に対する最も良いQ値を得る
        feature = next_player * next_board.reshape((-1))
        action_values = reg.predict([feature])[0]
        best_score = np.max(action_values)
    
        action_index = 64
        if not move.is_pass():
            action_index = move.y * 8 + move.x
    
        target = action_values.copy()
        target[action_index] = reward + gamma * best_score
        y.append(target)
    
        # 変数を更新
        player = next_player
        board = next_board

    # 1エピソードごとに回帰モデルを更新
    reg.partial_fit(X, y)

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
def move_by_random(env, player):
    """有効手の中からランダムに手を選ぶ"""
    moves = env.legal_moves(player)
    if len(moves) == 0:
        return Move.Pass(player)
    else:
        return np.random.choice(moves)

In [7]:
n_jobs = 4
n_episodes = 1000

def match():
    # ゲームのリセット
    env = othello.make()
    player, board = env.reset()

    # エピソード開始
    while not env.gameset():
        # 黒番:
        if player == Player.BLACK:
            # セル評価値が最も高い場所に着手する
            moves = env.legal_moves(player)

            if len(moves) == 0:
                move = Move.Pass(player)
            else:
                best_move = None
                best_score = -np.inf
                feature = player * board.reshape((-1))
                action_values = reg.predict([feature])[0]
                for m in moves:
                    action_index = m.y * 8 + m.x                    
                    if best_score < action_values[action_index]:
                        best_score = action_values[action_index]
                        best_move = m

                move = best_move
        # 白番:
        if player == Player.WHITE:
            # 着手可能な手があればランダムに1つを選ぶ
            move = move_by_random(env, player)

        # 着手による盤の状態の更新
        player, board = env.step(move)

    return env.count()


with tqdm_joblib(n_episodes):
    result = joblib.Parallel(n_jobs=n_jobs)(
        (joblib.delayed(match)() for _ in range(n_episodes)),
    )

result = np.array(result, dtype="int32")
b_win = np.sum(result[:, 0] > result[:, 1])
w_win = np.sum(result[:, 0] < result[:, 1])
draw = np.sum(result[:, 0] == result[:, 1])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
print(b_win, w_win, draw)

635 328 37


## 演習内容

今回の演習では以下のようなオセロをプレイするエージェントのクラスを実装する。ただし、基本的にはオセロ環境が渡されてくる`Agent.play`だけを編集すれば良い。

In [9]:
import numpy as np

class Agent(object):
    def __init__(self):
        pass

    def play(self, player, env) -> Move:
        """
        この関数を更新、以下はランダムに着手する例
        """
        moves = env.legal_moves()
        if len(moves) != 0:
            return Move.Pass(player)
        else:
            return np.random.choice(moves)

### 対戦相手のレベル

- **レベル1:** セル評価値を用いたアルファベータ探索で2手先まで読むAI
- **レベル2:** 上記のscikit-learnを用いた関数近似によって、次の手の価値を評価するAI
- **レベル3:** Q関数の学習方法を改良し、さらにQ関数に基づいて2手先まで読むAI

### ローカル環境でのテスト

### 本番環境でのテスト